In [0]:
EventHubConnectionString="Endpoint=sb://namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=Path=eventhub-name"


In [0]:
ehConf = {}
# For 2.3.15 version and above, the configuration dictionary requires that connection string be encrypted.
ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(EventHubConnectionString)

In [0]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"


jsonSchema = StructType([ StructField("timeStamp", TimestampType(), True), StructField("deviceId", StringType(), True), StructField("fleetId", StringType(), True),
                        StructField("batteryLevel", StringType(), True),StructField("current_lat", StringType(), True),StructField("current_lon", StringType(), True),
                        StructField("speed", StringType(), True)])



In [0]:
from pyspark.sql.functions import to_json,col,from_json
from pyspark.sql.functions import json_tuple
from pyspark.sql.functions import *  


df = spark.readStream.format("eventhubs").schema(jsonSchema).options(**ehConf).load()

df = df.withColumn("body", df["body"].cast("string"))


rawData = df.selectExpr("cast(body as string) as json").select(from_json("json", jsonSchema).alias("data")).select("data.*")

parsedData=rawData.select('timeStamp','deviceId','fleetId','batteryLevel','current_lat','current_lon','speed')  


streamingAvgSpeedDF = parsedData.groupBy(parsedData.deviceId,window(parsedData.timeStamp, "5 minute")).agg({'speed': 'avg'}).alias('average_speed')

display(streamingAvgSpeedDF)

deviceId,window,avg(speed)
de_000003,"List(2022-01-13T15:30:00.000+0000, 2022-01-13T15:35:00.000+0000)",42.325259399751786
de_000002,"List(2022-01-13T15:30:00.000+0000, 2022-01-13T15:35:00.000+0000)",58.2947217646496
de_000001,"List(2022-01-13T15:30:00.000+0000, 2022-01-13T15:35:00.000+0000)",33.81308433497934
